In [3]:
!pip install -q langchain groq llama-index chromadb sentence-transformers pypdf PyPDF2 langchain-community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.6/129.6 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.3/19.3 MB 87.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 304.2/304.2 kB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 69.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 64.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 95.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.3/267.3 kB 19.1 MB/s eta 0

In [101]:
from google.colab import userdata
import os
os.environ["groq_api"] = userdata.get('groq')
os.environ["groq_api"]

'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'

### **Step 1: Loading and Chunking the PDF Document**

##### ISLP_website - Copy.pdf : this document contains the ML and DL topics

In [56]:
from PyPDF2 import PdfReader

reader = PdfReader("ISLP_website - Copy.pdf")
raw_text = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        raw_text += text + "\n"

In [87]:
len(raw_text.split())

51521

In [82]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=300)
docs = splitter.create_documents([raw_text])
print(f'no of chunks is : {len(docs)}')

no of chunks is : 487


In [60]:
docs[450]

Document(metadata={}, page_content='426 10. Deep Learning\nModel # Parameters Mean Abs. Error Test Set R2\nLinear Regression 20 254.7 0.56\nLasso 12 252.3 0.51\nNeural Network 1345 257.4 0.54\nTABLE 10.2. Prediction results on the Hitters test data for linear models fit\nby ordinary least squares and lasso, compared to a neural network fit by stochastic\ngradient descent with dropout regularization.\nCoefficient Std. error t-statistic p-value\nIntercept -226.67 86.26 -2.63 0.0103\nHits 3.06 1.02 3.00 0.0036\nWalks 0.181 2.04 0.09 0.9294\nCRuns 0.859 0.12 7.09 <0.0001\nPutOuts 0.465 0.13 3.60 0.0005\nTABLE 10.3. Least squares coefficient estimates associated with the regres-\nsion ofSalaryon four variables chosen by lasso on the Hitters data set. This\nmodel achieved the best performance on the test data, with a mean absolute error\nof 224.8. The results reported here were obtained from a regression on the test\ndata, which was not used in fitting the lasso model.\nTable10.2compares the

In [61]:
docs[451]

Document(metadata={}, page_content='model achieved the best performance on the test data, with a mean absolute error\nof 224.8. The results reported here were obtained from a regression on the test\ndata, which was not used in fitting the lasso model.\nTable10.2compares the results. We see similar performance for all three\nmodels. We report the mean absolute error on the test data, as well as\nthe test R2for each method, which are all respectable (see Exercise 5).\nWe spent a fair bit of time fiddling with the configuration parameters of\nthe neural network to achieve these results. It is possible that if we were to\nspend more time, and got the form and amount of regularization just right,\nthat we might be able to match or even outperform linear regression and\nthe lasso. But with great ease we obtained linear models that work well.\nLinear models are much easier to present and understand than the neural\nnetwork, which is essentially a black box. The lasso selected 12 of the 19')

##### To handle large context windows, I used the RecursiveCharacterTextSplitter to break the document into overlapping chunks of 1000 characters each and used overlap of 300 characters.

##### LangChain's Recursive Character Text Splitter :
* It is used to split long texts into smaller chunks while preserving meaning as much as possible.
* It tries to break text at logical boundaries such as paragraphs, sentences, or words—recursively using a list of separators like [comma , fullstop , \n]



### **Step 2: Embeding the chunks (data) & Storing Chunks in Vector/**

In [89]:
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
import shutil

In [90]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma

embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

vector_store = Chroma.from_documents(documents= docs, embedding= embedding_model, persist_directory="./chroma_index")
vector_store.persist()

In [91]:
shutil.make_archive('chroma_index', 'zip', './chroma_index')

'/content/chroma_index.zip'

### **Step 3 : Loading Groq LLM and QA model**

In [92]:
from langchain.llms import Groq

llm = Groq(
    model="llama3-8b-8192",
    api_key=os.getenv("groq_api")
)

ImportError: cannot import name 'Groq' from 'langchain.llms' (/usr/local/lib/python3.11/dist-packages/langchain/llms/__init__.py)

#### ERROR : ImportError: cannot import name 'Groq' from 'langchain.llms'

##### So using ChatOpenAI to connect with Groq's OpenAI-compatible API endpoint for LLM inference

In [68]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA

llm = ChatOpenAI(
    model= "llama-3.3-70b-versatile",
    # model = 'llama3-8b-8192',
    temperature= 0,
    openai_api_key= os.getenv("groq_api"),
    openai_api_base= "https://api.groq.com/openai/v1")

qa_chain = RetrievalQA.from_chain_type(
    llm= llm,
    retriever= vector_store.as_retriever(search_kwargs={"k": 3}),
    return_source_documents= True)

* I used Groq AI's 'llama-3.3-70b-versatile' model as the core LLM, leveraging its ultra-fast inference.
* Retrieval-Augmented Generation (RAG) was implemented using ChromaDB and semantic embeddings to dynamically fetch relevant chunks and provide contextual answers.

In [69]:
query = "What Generative Models for Classification ?"
response = qa_chain(query)

In [70]:
from IPython.display import Markdown
Markdown(response["result"])

According to the text, Generative Models for Classification involve modeling the distribution of the predictors X separately in each of the response classes, and then using Bayes' theorem to estimate the probabilities Pr(Y=k|X=x). 

In other words, instead of directly modeling the conditional distribution of the response Y given the predictor(s) X, generative models model the joint distribution of X and Y by estimating the distribution of X in each class, and then use Bayes' theorem to obtain the conditional distribution of Y given X.

Some examples of generative models for classification mentioned in the text are:

1. Linear Discriminant Analysis
2. Quadratic Discriminant Analysis
3. Naive Bayes 

These models are called "generative" because they model the underlying process that generates the data, rather than just modeling the conditional distribution of the response variable given the predictors.

#### Source references

In [71]:
for i, doc in enumerate(response["source_documents"], 1):
    print(f"--- Source {i} ---")
    print(doc.page_content[:300])
    print()

--- Source 1 ---
4
Classification
The linear regression model discussed in Chapter 3assumes that the re-
sponse variable Yis quantitative. But in many situations, the response
variable is instead qualitative . For example, eye color is qualitative. Of-qualitativeten qualitative variables are referred to as categoric

--- Source 2 ---
or class. On the other hand, often the methods used for classification first
predict the probability that the observation belongs to each of the cate-
gories of a qualitative variable, as the basis for making the classification.
In this sense they also behave like regression methods.
There are many 

--- Source 3 ---
146 4. Classification
Thus, rather than estimating coefficients for K−1classes, we actually
estimate coefficients for all Kclasses. It is not hard to see that as a result
of (4.13), the log odds ratio between the kth andk/primeth classes equals
log/parenleftbiggPr(Y=k|X=x)
Pr(Y=k/prime|X=x)/parenrig



### **multi-turn conversation**

In [72]:
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain

memory = ConversationBufferMemory(memory_key= "chat_history", return_messages= True)

chat_chain = ConversationalRetrievalChain.from_llm(llm= llm,retriever= vector_store.as_retriever(),memory= memory)

In [73]:
query2 = "What are Linear Discriminant Analysis ?"
chat_response2 = chat_chain.run(query2)
Markdown(chat_response2)

Linear Discriminant Analysis (LDA) is a classification technique that assumes the observations within each class are drawn from a multivariate Gaussian distribution with a class-specific mean vector and a covariance matrix that is common to all classes. It is used to predict a qualitative response by assigning an observation to the class for which a linear function of the observation is largest.

In LDA, the classifier results from assuming that the observations from each class are drawn from a Gaussian distribution, and plugging estimates for the parameters into Bayes' theorem in order to perform predictions. The LDA classifier assigns an observation X = x to the class for which a linear function, known as the discriminant function, is largest.

The discriminant function in LDA is a linear function of the observation, which is why it is called "linear" discriminant analysis. The decision boundary in LDA is a linear line that separates the classes, and all points to one side of the line are assigned to one class, while points to the other side are assigned to another class.

In [74]:
query3 = "what is the difference between the LDA and the QDA?"
chat_response3 = chat_chain.run(query3)
Markdown(chat_response3)

The main difference between Linear Discriminant Analysis (LDA) and Quadratic Discriminant Analysis (QDA) is the assumption they make about the covariance matrix of the classes.

LDA assumes that all classes share a common covariance matrix, which means that the variance and covariance of the features are the same across all classes. This assumption leads to a linear decision boundary.

On the other hand, QDA assumes that each class has its own covariance matrix, which means that the variance and covariance of the features can differ between classes. This assumption leads to a quadratic decision boundary.

In other words, LDA is suitable when the classes have similar covariance structures, while QDA is more flexible and can handle classes with different covariance structures. However, QDA can suffer from higher variance and requires a larger training set to estimate the class-specific covariance matrices accurately.

#### Handle Ambiguous or Vague Queries

In [75]:
query4 = "what is the GEN AI"
chat_response4 = chat_chain.run(query4)
Markdown(chat_response4)

I don't know what the "GEN AI" refers to. The provided context does not mention "GEN AI", and I couldn't find any relevant information about it. If you could provide more context or clarify what you mean by "GEN AI", I'll do my best to help.

##### Summarization

In [100]:
from langchain.chains.summarize import load_summarize_chain
summarizer = load_summarize_chain(llm, chain_type="stuff")

summary = summarizer.run(docs[:5])
summary

'Here is a concise summary:\n\nThe book "An Introduction to Statistical Learning, With Applications in Python" (ISLP) is an introduction to statistical learning, a set of tools for making sense of complex datasets. The book is a Python-based alternative to the popular book "An Introduction to Statistical Learning, With Applications in R" (ISLR). ISLP covers the same materials as ISLR, but with labs implemented in Python. The book is intended for advanced undergraduates or master\'s students in statistics or related quantitative fields, or for individuals in other disciplines who want to use statistical learning tools to analyze their data.'

#### Using different model [ llama3-8b-8192 ]

In [76]:
llm = ChatOpenAI(
    # model="llama-3.3-70b-versatile",
    model = 'llama3-8b-8192',
    temperature=0,
    openai_api_key=os.getenv("groq_api"),
    openai_api_base="https://api.groq.com/openai/v1")

qa_chain = RetrievalQA.from_chain_type(llm = llm,retriever = vector_store.as_retriever(search_kwargs={"k": 3}),return_source_documents = True)

In [77]:
query = "What Generative Models for Classification ?"
response = qa_chain(query)
Markdown(response["result"])

According to the provided context, Generative Models for Classification are an alternative approach to estimating the probabilities Pr(Y=k|X=x) in logistic regression. In this approach, the distribution of the predictors X is modeled separately in each of the response classes (i.e., for each value of Y), and then Bayes' theorem is used to flip these around into estimates for Pr(Y=k|X=x).

In [78]:
query = "what is the difference between the Multiple Logistic Regression and Multinomial Logistic Regression ?"
response = qa_chain(query)
Markdown(response["result"])

According to the provided context, Multiple Logistic Regression and Multinomial Logistic Regression are two different types of logistic regression models.

Multiple Logistic Regression is a type of logistic regression that allows for more than two classes for the response variable, but it treats each class as a separate outcome. This means that the model estimates the probability of each class separately, and the coefficients are interpreted as the change in the log odds of each class compared to a baseline class.

On the other hand, Multinomial Logistic Regression is a type of logistic regression that also allows for more than two classes for the response variable, but it estimates the probability of each class relative to a baseline class. The coefficients in a multinomial logistic regression model are interpreted as the change in the log odds of each class compared to the baseline class.

In other words, Multiple Logistic Regression estimates the probability of each class separately, while Multinomial Logistic Regression estimates the probability of each class relative to a baseline class.